In [26]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['화자 관계 식별', '화자 감정 식별']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess

pass

#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            data = {'input': {}, 'output': {}}
            data['input']['info'] = {}
            data['input']['info']['id'] = line['info']['id']
            data['input']['info']['situation'] = line['info']['situation']
            data['input']['info']['speaker_emotion'] = line['info']['speaker_emotion']
            
            #### data preprocess end 


            #### 화자 관계 식별
            # data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input']['utterances'] = []
            for utterance in line['utterances']:
                utterances = {}
                utterances['utterance_id'] = utterance['utterance_id']
                utterances['role'] = utterance['role']
                utterances['text'] = utterance['text']
                data['input']['utterances'].append(utterances)
            data['output']['relation'] = line['info']['relation']

            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 관계 식별 end


            #### 화자 감정 식별
            # data = {'input': None, 'output': None}
            ## preprocess data from line
            del data['input']['info']['speaker_emotion']
            del data['output']['relation']
            data['input']['info']['relation'] = line['info']['relation']
            data['input']['info']['speaker_relation'] = line['info']['speaker_relation']
            data['output']['emotion'] = []
            check = 0
            for utterance in line['utterances']:
                speaker_emotion = {}
                listener_emotion = {}
                if utterance['role'] == 'speaker':
                    speaker_emotion['utterance_id'] = utterance['utterance_id']
                    speaker_emotion['role'] = utterance['role']
                    if utterance['speaker_changeEmotion'] == None and check == 0:
                         speaker_emotion['speaker_emotion'] = [line['info']['speaker_emotion']]
                    elif utterance['speaker_changeEmotion'] == '중립' or check == 1:
                        speaker_emotion['speaker_emotion'] = ['중립']
                        check = 1
                    data['output']['emotion'].append(speaker_emotion)
                    
                elif utterance['role'] == 'listener':
                    listener_emotion['utterance_id'] = utterance['utterance_id']
                    listener_emotion['role'] = utterance['role']
                    if utterance['listener_empathy'] == None:
                        listener_emotion['listener_emotion'] = None
                    elif utterance['listener_empathy'] != None:
                        if not isinstance(utterance['listener_empathy'], list):
                            listener_emotion['listener_emotion'] = [utterance['listener_empathy']]
                        else : listener_emotion['listener_emotion'] = utterance['listener_empathy']
                    data['output']['emotion'].append(listener_emotion)
            
                
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 화자 감정 식별 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 42/42 [00:00<00:00, 73.27it/s]
